<a href="https://colab.research.google.com/github/ara42/Deep-Learning/blob/main/%ED%85%8D%EC%8A%A4%ED%8A%B8_%EC%B2%98%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.2M  100 80.2M    0     0  7897k      0  0:00:10  0:00:10 --:--:-- 17.2M


In [2]:
!tar -xf aclImdb_v1.tar.gz

In [3]:
!rm -r aclImdb/train/unsup

In [4]:
import os ,pathlib, shutil,random

In [5]:
base_dir = pathlib.Path('aclImdb')
val_dir = base_dir/"val"
train_dir = base_dir/"train"
for category in ("neg","pos"):
  os.makedirs(val_dir/category)
  files = os.listdir(train_dir/category)
  random.Random(1337).shuffle(files)
  num_val_samples = int(0.2*len(files))
  val_files = files[:num_val_samples]
  for fname in val_files:
    shutil.move(train_dir/category/fname,
                val_dir/category/fname)

In [6]:
from tensorflow import keras

In [7]:
train_ds = keras.utils.text_dataset_from_directory('aclImdb/train',
                                                   batch_size=32)
val_ds = keras.utils.text_dataset_from_directory('aclImdb/val',
                                                   batch_size=32)
test_ds = keras.utils.text_dataset_from_directory('aclImdb/test',
                                                   batch_size=32)

Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.


## 시퀀스

In [8]:
from keras import layers

In [9]:
max_length = 600
max_tokens = 20000

In [10]:
tv = layers.TextVectorization(
    max_tokens=max_tokens,
    output_mode="int",
    output_sequence_length=max_length,
)
text_only_train_ds = train_ds.map(lambda x,y: x)

tv.adapt(text_only_train_ds) #vacabulary 형성

In [11]:
int_train_ds = train_ds.map(
    lambda x,y: (tv(x),y),
    num_parallel_calls=4
)
int_val_ds = val_ds.map(
    lambda x,y: (tv(x),y),
    num_parallel_calls=4
)
int_test_ds = test_ds.map(
    lambda x,y: (tv(x),y),
    num_parallel_calls=4
)

In [12]:
import tensorflow as tf

In [13]:
inputs = keras.Input(shape=(None,),dtype='int64')
embedded = tf.one_hot(inputs,depth=max_tokens)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1,activation='sigmoid')(x)
model = keras.Model(inputs,outputs)
model.compile(loss=keras.losses.binary_crossentropy,
              metrics=['accuracy'])

In [ ]:
#RAM 부족(실행X)
mcp = keras.callbacks.ModelCheckpoint('one_hot_bidir_lstm.keras',
                                      save_best_only=True)
model.fit(int_train_ds.cache(),
          validation_data=int_val_ds.cache(),
          epochs=10,
          callbacks=[mcp])
model = keras.models.load_model('one_hot_bidir_lstm.keras')
model.evaluate(int_test_ds)

In [13]:
inputs = keras.Input(shape=(None,),dtype='int64')
embedded = layers.Embedding(input_dim=max_tokens, output_dim=256)(inputs)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1,activation='sigmoid')(x)
model = keras.Model(inputs,outputs)
model.compile(loss=keras.losses.binary_crossentropy,
              metrics=['accuracy'])

In [14]:
mcp = keras.callbacks.ModelCheckpoint('embedding_bidir_lstm.keras',
                                      save_best_only=True)
model.fit(int_train_ds.cache(),
          validation_data=int_val_ds.cache(),
          epochs=10,
          callbacks=[mcp])
model = keras.models.load_model('embedding_bidir_lstm.keras')
model.evaluate(int_test_ds)

Epoch 1/10
625/625 [==============================] - 109s 160ms/step - loss: 0.5176 - accuracy: 0.7452 - val_loss: 0.4146 - val_accuracy: 0.8308
Epoch 2/10
625/625 [==============================] - 23s 37ms/step - loss: 0.3582 - accuracy: 0.8662 - val_loss: 0.3340 - val_accuracy: 0.8664
Epoch 3/10
625/625 [==============================] - 23s 36ms/step - loss: 0.2871 - accuracy: 0.8946 - val_loss: 0.3223 - val_accuracy: 0.8800
Epoch 4/10
625/625 [==============================] - 22s 36ms/step - loss: 0.2393 - accuracy: 0.9172 - val_loss: 0.3881 - val_accuracy: 0.8608
Epoch 5/10
625/625 [==============================] - 21s 34ms/step - loss: 0.2123 - accuracy: 0.9304 - val_loss: 0.3778 - val_accuracy: 0.8734
Epoch 6/10
625/625 [==============================] - 22s 35ms/step - loss: 0.1780 - accuracy: 0.9427 - val_loss: 0.3813 - val_accuracy: 0.8740
Epoch 7/10
625/625 [==============================] - 25s 40ms/step - loss: 0.1740 - accuracy: 0.9437 - val_loss: 0.4722 - val_accurac

[0.3482641279697418, 0.8660399913787842]